In [ ]:
import os
import PyPDF2
import openai
import requests

from library.exportation import export_prompt_response, export_article

In [1]:
# test ollama
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': """Please follow the format to answer the questions.
    give me the answer maxmium to 3 sentense.
       
    Here is the retrieved context:
terms of the connected transactions conducted during the Reporting Period. During the Reporting Period, the aggregate revenue 
amount of the Continuing Connected Transactions at Issuer Level and the Continuing Connected Transactions at Subsidiary 
Level was HK$6,802.2 million and HK$429.7 million, respectively, accounting for approximately 78.1% and 4.9% of our total 
revenue, respectively, during the Reporting Period. The total procurement of the Continuing Connected Transactions at Issuer 
Level and the Continuing Connected Transactions at Subsidiary Level was HK$1,635.2 million and HK$1,583.3 million, 
respectively, accounting for approximately 19.0% and 18.4% of our total purchase, respectively, during the Reporting Period.
The details of the Continuing Connected Transactions conducted by the Group during the Reporting Period that are subject to 
reporting requirement under the Listing Rules are set out in this section. Unless otherwise defined herein, capitalised terms used
decrease in sales volume was mainly due to that marketable goods were in short supply and could not fully meet customer needs. 
The increase in revenue while decrease in gross profit are mainly because there have been significant changes in product structure 
compared with the same period last year, namely, the sales volume of finished tobacco strips with higher unit sale price but lower 
gross profit level has shown a relatively significant increase in sales proportion, while sales volume of tobacco leaf by-products 
with higher gross profit level but lower unit sale price has decreased in sales proportion.
PROSPECTS FOR THE SECOND HALF OF 2024
In the second half of 2024, we will uphold the tenet of “respect market, respect rules, respect investors”, and based on the 
strategic position of “capital markets operation and international business expansion platform” to achieve the Group’s 
high-quality development goals by prioritising the following areas in our efforts:
Notes:
1. In light of the fact that CNTC directly controls one third or more of the voting rights in the shareholders’ meetings of CTIG, in accordance 
with the SFO, the interests of CTIG are deemed to be, and have therefore been included in, the interests of CNTC.
2. As at 30 June 2024, the Company had 691,680,000 Shares in issue.
Apart from the foregoing, as at 30 June 2024, no other person (other than a Director or the chief executive of the Company) 
had any interests or short positions in the Shares and underlying Shares as recorded in the register required to be kept by the 
Company under section 336 of the SFO, or as otherwise notified to the Company and the Stock Exchange.41
Interim Report 2024
Other Information (Continued)
CHANGES IN DIRECTORS’ BIOGRAPHICAL DETAILS
During the annual general meeting of the Company dated 17 May 2024 (the “2024 AGM”), Mr. Dai Jiahui was re-elected as
Question: What is the total revenue for the reporting period? 
Answer: """,
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The total revenue for the Reporting Period from Continuing Connected Transactions at Issuer Level and Subsidiary Level was HK$6,802.2 million and HK$429.7 million, respectively, accounting for approximately 78.1% and 4.9% of our total revenue, respectively.
The total revenue for the Reporting Period from Continuing Connected Transactions at Issuer Level and Subsidiary Level was HK$6,802.2 million and HK$429.7 million, respectively, accounting for approximately 78.1% and 4.9% of our total revenue, respectively.


In [ ]:
def load_pdf_text(pdf_path):
    """Load text from a PDF file."""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

def generate_answer(context, question, api_key):
    model = 'gpt-4o-mini'
    url = "https://api.ohmygpt.com/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a financial assistant."},
            {"role": "user", "content": f"Context: {context} Question: {question}"}
        ],
        "temperature": 0.7
    }
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    return response_json["choices"][0]["message"]["content"]

def generate_summary(answers, api_key):
    model = 'gpt-4o-mini'
    url = "https://api.ohmygpt.com/v1/chat/completions"

    summary_prompt = f"""Based on the questions and answers provided. please write a financial report. \n\nThe context is as below \n{answers}"""
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a article writer."},
            {"role": "user", "content": summary_prompt}
        ],
        "temperature": 0.7
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"]


In [ ]:
api_key = ""
file_name = '6055.HK'
folder_name = '3_data'
pdf_file_path = os.path.join(folder_name, f'{file_name}.pdf')
pdf_text = load_pdf_text(pdf_file_path)

In [ ]:
prompt = prompt()
answers = generate_answer(pdf_text, prompt, api_key)
aritcle = generate_summary(answers, api_key)

In [ ]:
export_article(file_name, aritcle)

report has been exported to 2_article_log\6055.HK_20241122_0016.txt


In [ ]:
# rag_pipeline.py

import requests
from prompt_template import prompt_template
import langchain as lc
from langchain.document_loaders import LocalLoader
from langchain.retrievers import DenseRetriever
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize document loader
loader = LocalLoader(
    path="path/to/your/documents",
    file_extensions=[".txt", ".pdf", ".docx"]
)

# Initialize retriever
retriever = DenseRetriever(
    loader=loader,
    model_name="sentence-transformers/all-mpnet-base-v2"
)

# Load the pre-trained sentence transformer model
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def chunk_text(text, chunk_size=512, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

def embed_text_chunks(chunks):
    embeddings = embedding_model.encode(chunks, convert_to_tensor=True)
    return embeddings

def get_most_relevant_chunk(question, chunks, embeddings):
    question_embedding = embedding_model.encode([question], convert_to_tensor=True)
    similarity_scores = cosine_similarity(question_embedding, embeddings)
    most_relevant_idx = np.argmax(similarity_scores)
    return chunks[most_relevant_idx]

def generate_response(context, question, api_key):
    model = 'ollama'
    url = "https://api.ollama.ai/v1/chat/completions"
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    
    prompt = prompt_template.format(context=context, question=question)
    
    data = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a knowledgeable assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7
    }
    
    response = requests.post(url, headers=headers, json=data)
    response_json = response.json()
    return response_json["choices"][0]["message"]["content"]

# Main function to load PDF, chunk text, embed chunks, and generate response
def main(pdf_path, question, api_key):
    # Load text from PDF (assuming a function load_pdf_text is defined)
    text = load_pdf_text(pdf_path)
    chunks = chunk_text(text)
    
    chunk_embeddings = embed_text_chunks(chunks)
    most_relevant_chunk = get_most_relevant_chunk(question, chunks, chunk_embeddings)
    
    answer = generate_response(most_relevant_chunk, question, api_key)
    print(answer)

# Example usage
if __name__ == "__main__":
    pdf_path = "path/to/your/pdf_file.pdf"
    question = "What are the recent advancements in AI research?"
    api_key = "your_api_key"
    
    main(pdf_path, question, api_key)
